# Kalman Filter Implementation for Position Estimation

This notebook demonstrates a practical implementation of the Kalman filter for estimating the position and velocity of an object moving with constant velocity, using noisy position measurements.

## Introduction

The Kalman filter is an optimal recursive algorithm for estimating the state of a linear dynamic system from a series of noisy measurements. It's widely used in engineering applications such as:
- Vehicle tracking and navigation
- Sensor fusion
- Control systems
- Signal processing

## Problem Setup

**State Vector:**  
We want to estimate the object's position and velocity:

$$\mathbf{x}_k = \begin{bmatrix} \text{position}_k \\ \text{velocity}_k \end{bmatrix}$$

**Measurement:**  
We only have access to noisy **position** measurements.

**Model:**  
We assume the object moves with **constant velocity** (with some process noise).

## 1. Kalman Filter Setup and Matrix Definitions

### State Transition Model
For an object moving with constant velocity, the state transition follows:

$$\mathbf{x}_k = \mathbf{F}_k \mathbf{x}_{k-1} + \mathbf{w}_{k-1}$$

Where:
- **F**: State transition matrix (describes how the state evolves from time k-1 to k)
- **w**: Process noise (uncertainty in our motion model)

### Measurement Model
We observe only the position with noise:

$$\mathbf{z}_k = \mathbf{H}_k \mathbf{x}_k + \mathbf{v}_k$$

Where:
- **H**: Observation matrix (maps state to measurements)
- **v**: Measurement noise

### Noise Covariance Matrices
- **Q**: Process noise covariance (how much we trust our motion model)
- **R**: Measurement noise covariance (how much we trust our measurements)
- **P**: State estimate covariance (uncertainty in our current estimate)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate motion with constant velocity and noisy position measurements

# Time step
dt = 1.0

# State transition matrix (constant velocity model)
F = np.array([[1, dt],
              [0, 1]])

# Observation matrix (we only measure the position)
H = np.array([[1, 0]])

# Process noise covariance matrix (uncertainty in model prediction)
Q = np.array([[1, 0],
              [0, 1]])

# Measurement noise covariance matrix (uncertainty in measurement)
R = np.array([[10]])

# Initial state estimate (position and velocity)
x = np.array([[0],  # Initial position
              [1]])  # Initial velocity

# Initial estimate covariance matrix
P = np.array([[500, 0], 
              [0, 500]])

# Number of iterations (time steps)
n = 50

## 2. Generate Simulation Data

Now we'll create the ground truth data and simulate noisy measurements. In a real application, you would replace this simulation with actual sensor data.

### What we're simulating:
- **True trajectory**: Object moving with constant velocity (1 unit/time step)
- **Noisy measurements**: Position measurements corrupted with Gaussian noise
- The measurement noise has standard deviation = √10 ≈ 3.16 units

In [ ]:
# Simulated true positions and noisy measurements
true_positions = []
measurements = []
velocities = []
position = 0
velocity = 1
for i in range(n):
    # True position and velocity (without noise)
    position += velocity * dt
    true_positions.append(position)
    
    # Noisy measurement (position)
    noisy_measurement = position + np.random.normal(0, np.sqrt(R[0, 0]))
    measurements.append(noisy_measurement)


## 3. Kalman Filter Algorithm

The Kalman filter operates in two main steps that repeat for each time step:

### Step 1: Prediction (Time Update)
- **State Prediction**: $\hat{\mathbf{x}}_{k|k-1} = \mathbf{F}_k \hat{\mathbf{x}}_{k-1|k-1}$
- **Covariance Prediction**: $\mathbf{P}_{k|k-1} = \mathbf{F}_k \mathbf{P}_{k-1|k-1} \mathbf{F}_k^T + \mathbf{Q}_k$

### Step 2: Update (Measurement Update)
- **Innovation**: $\mathbf{y}_k = \mathbf{z}_k - \mathbf{H}_k \hat{\mathbf{x}}_{k|k-1}$
- **Innovation Covariance**: $\mathbf{S}_k = \mathbf{H}_k \mathbf{P}_{k|k-1} \mathbf{H}_k^T + \mathbf{R}_k$
- **Kalman Gain**: $\mathbf{K}_k = \mathbf{P}_{k|k-1} \mathbf{H}_k^T \mathbf{S}_k^{-1}$
- **State Update**: $\hat{\mathbf{x}}_{k|k} = \hat{\mathbf{x}}_{k|k-1} + \mathbf{K}_k \mathbf{y}_k$
- **Covariance Update**: $\mathbf{P}_{k|k} = (\mathbf{I} - \mathbf{K}_k \mathbf{H}_k) \mathbf{P}_{k|k-1}$

In [ ]:
# Kalman filter process
estimates = []  # Store position estimates
for i in range(n):
    # Prediction Step
    x = np.dot(F, x)  # State prediction
    P = np.dot(np.dot(F, P), F.T) + Q  # Covariance prediction
    
    # Measurement update step (if we have a measurement)
    z = np.array([[measurements[i]]])  # Current measurement (noisy position)
    y = z - np.dot(H, x)  # Innovation (residual)
    S = np.dot(H, np.dot(P, H.T)) + R  # Innovation covariance
    K = np.dot(np.dot(P, H.T), np.linalg.inv(S))  # Kalman gain
    
    x = x + np.dot(K, y)  # Updated state estimate
    P = np.dot(np.eye(2) - np.dot(K, H), P)  # Updated covariance estimate
    
    # Store the estimate
    estimates.append(x[0, 0])

## 4. Results Visualization

Let's plot the results to see how well the Kalman filter performs:

- **Blue line**: Kalman filter estimates (our filtered output)
- **Red dots**: Noisy measurements (what we actually observe)
- **Orange line**: True position (ground truth, unknown in practice)

**Expected behavior:**
The Kalman filter should provide estimates that are much smoother than the noisy measurements and closely track the true position.

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.plot(true_positions, label='True Position')
plt.plot(measurements, 'ro', label='Noisy Measurements')
plt.plot(estimates, 'b-', label='Kalman Filter Estimate')
plt.legend()
plt.xlabel('Time Step')
plt.ylabel('Position')
plt.title('Kalman Filter for Position Estimation')
plt.show()

## 5. Key Insights and Analysis

### What the Kalman Filter Accomplished:

1. **Noise Reduction**: The filter significantly reduced the measurement noise while preserving the underlying signal trend.

2. **State Estimation**: Even though we only measured position, the filter also estimated velocity internally, which helped improve position predictions.

3. **Optimal Balance**: The Kalman gain automatically balances between:
   - Trust in the prediction (based on process noise Q)
   - Trust in the measurement (based on measurement noise R)

### Parameter Tuning Effects:

- **Larger Q**: More trust in measurements, faster adaptation to changes
- **Larger R**: More trust in predictions, smoother but potentially slower tracking
- **Initial P**: Affects convergence speed (high uncertainty = faster initial adaptation)

### Real-World Applications:
- GPS navigation systems
- Robot localization
- Target tracking in radar systems
- Economic forecasting
- Battery state estimation

In [ ]:
## 6. Exercises and Extensions

### Try These Modifications:

1. **Change the noise levels**: 
   - Modify `Q` and `R` matrices to see how they affect performance
   - Try `R = [[100]]` for very noisy measurements
   - Try `Q = [[0.1, 0], [0, 0.1]]` for more trust in the model

2. **Different motion models**:
   - Add acceleration: extend state to [position, velocity, acceleration]
   - Try non-constant velocity: modify the true motion in the simulation

3. **Missing measurements**:
   - Skip some measurements (set them to `None`) and see how the filter handles gaps

4. **Multiple sensors**:
   - Add velocity measurements to the observation model

### Mathematical Foundation:

The Kalman filter is optimal (in the least squares sense) for linear systems with Gaussian noise. For nonlinear systems, consider:
- **Extended Kalman Filter (EKF)**: Linearizes nonlinear functions
- **Unscented Kalman Filter (UKF)**: Uses sigma points for better nonlinear handling
- **Particle Filters**: For highly nonlinear or non-Gaussian systems